In [ ]:
from xtb.interface import Calculator, Param, Solvent
import periodictable as pt
from scipy.constants import physical_constants


ANGST_IN_BOHR = physical_constants['Bohr radius'][0] * 1.0e10

def xtb_energy(protein, rotlib):
    coords = protein.atoms.positions[~rotlib.clash_ignore_idx]
    coords = np.concatenate([coords, rotlib.coords[0]])
    
    elements = protein.atoms.types[~rotlib.clash_ignore_idx]
    elements = np.concatenate([elements, rotlib.atom_types])

    atom_numbers = np.array([getattr(pt, e).number for e in elements])

    xtb_calculator = Calculator(Param.GFN2xTB, atom_numbers, coords / ANGST_IN_BOHR, uhf=1, charge=)
    xtb_calculator.set_solvent(Solvent.h2o)

    # Claculate energy of each state
    E = []
    for rot in rotlib.coords:
        coords[-len(rot):] = rot
        xtb_calculator.update(coords/ANGST_IN_BOHR)
        E.append(xtb_calculator.singlepoint().get_energy())
        
    # Convert to kcal/mol and reduce to relative difference in energy
    E = np.array(E) * 630
    E -= min(E)
    
    
    return E